<a href="https://colab.research.google.com/github/vitthal-bhandari/Homophobia-Transphobia-Detection/blob/master/Ablations/Ablation_studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initializing environment

In [1]:
#check gpu usage
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr  8 20:18:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#check ram usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install datasets
!pip install transformers
!pip3 install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 212 kB 61.1 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 136 kB 56.3 MB/s 
     |████████████████████████████████| 1.1 MB 62.5 MB/s 
     |████████████████████████████████| 127 kB 75.8 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 271 kB 61.0 MB/s 
     |████████████████████████████████| 144 kB 70.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 4.2 MB/s 


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np
import torch

# Reading pre-processed data

## English

In [5]:
#Reading untouched data (non-preprocessed, non-augmented)

eng_train_nonprepro_nonaug_dataset = load_dataset("csv", data_files="eng_train_nonprepro_nonaug.csv")
eng_train_nonprepro_nonaug = eng_train_nonprepro_nonaug_dataset["train"]

eng_dev_nonprepro_nonaug_dataset = load_dataset("csv", data_files="eng_dev_nonprepro_nonaug.csv")
eng_dev_nonprepro_nonaug = eng_dev_nonprepro_nonaug_dataset["train"]

eng_test_nonprepro_nonaug_dataset = load_dataset("csv", data_files="eng_test_nonprepro_nonaug.csv")
eng_test_nonprepro_nonaug = eng_test_nonprepro_nonaug_dataset["train"]

print(len(eng_train_nonprepro_nonaug), len(eng_dev_nonprepro_nonaug), len(eng_test_nonprepro_nonaug))

Using custom data configuration default-2cdd479e3759a38e


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2cdd479e3759a38e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-44d3011cbee4785e


...

Using custom data configuration default-77895f949bf59e82


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-77895f949bf59e82/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

3162 791 990


In [6]:
#Reading preprocessed data (non-augmented)

eng_train_prepro_nonaug_dataset = load_dataset("csv", data_files="eng_train_prepro_nonaug.csv")
eng_train_prepro_nonaug = eng_train_prepro_nonaug_dataset["train"]

eng_dev_prepro_nonaug_dataset = load_dataset("csv", data_files="eng_dev_prepro_nonaug.csv")
eng_dev_prepro_nonaug = eng_dev_prepro_nonaug_dataset["train"]

eng_test_prepro_nonaug_dataset = load_dataset("csv", data_files="eng_test_prepro_nonaug.csv")
eng_test_prepro_nonaug = eng_test_prepro_nonaug_dataset["train"]

print(len(eng_train_prepro_nonaug), len(eng_dev_prepro_nonaug), len(eng_test_prepro_nonaug))

Using custom data configuration default-a344e4dcee9dd682


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a344e4dcee9dd682/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-524257c7b45f2840


...

Using custom data configuration default-4734744f538099f6


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4734744f538099f6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

3162 791 990


In [7]:
#Reading augmented data (non-preprocessed)

eng_train_nonprepro_aug_dataset = load_dataset("csv", data_files="eng_train_nonprepro_aug.csv")
eng_train_nonprepro_aug = eng_train_nonprepro_aug_dataset["train"]

eng_dev_nonprepro_aug_dataset = load_dataset("csv", data_files="eng_dev_nonprepro_aug.csv")
eng_dev_nonprepro_aug = eng_dev_nonprepro_aug_dataset["train"]

eng_test_nonprepro_aug_dataset = load_dataset("csv", data_files="eng_test_nonprepro_aug.csv")
eng_test_nonprepro_aug = eng_test_nonprepro_aug_dataset["train"]

print(len(eng_train_nonprepro_aug), len(eng_dev_nonprepro_aug), len(eng_test_nonprepro_aug))

Using custom data configuration default-407ffc2477098529


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-407ffc2477098529/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-bf2a7c86bda0920e


...

Using custom data configuration default-f91701a2a008354c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f91701a2a008354c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

4530 791 990


## Tamil

In [8]:
#Reading untouched data (non-preprocessed, non-augmented)

tam_train_nonprepro_nonaug_dataset = load_dataset("csv", data_files="tam_train_nonprepro_nonaug.csv")
tam_train_nonprepro_nonaug = tam_train_nonprepro_nonaug_dataset["train"]

tam_dev_nonprepro_nonaug_dataset = load_dataset("csv", data_files="tam_dev_nonprepro_nonaug.csv")
tam_dev_nonprepro_nonaug = tam_dev_nonprepro_nonaug_dataset["train"]

tam_test_nonprepro_nonaug_dataset = load_dataset("csv", data_files="tam_test_nonprepro_nonaug.csv")
tam_test_nonprepro_nonaug = tam_test_nonprepro_nonaug_dataset["train"]

print(len(tam_train_nonprepro_nonaug), len(tam_dev_nonprepro_nonaug), len(tam_test_nonprepro_nonaug))

Using custom data configuration default-b953d96a6dcbb3c5


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b953d96a6dcbb3c5/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-bc112aabfe00cc0a


...

Using custom data configuration default-b90d4dcc6b655e73


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b90d4dcc6b655e73/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

2655 665 829


In [9]:
#Reading preprocessed data (non-augmented)

tam_train_prepro_nonaug_dataset = load_dataset("csv", data_files="tam_train_prepro_nonaug.csv")
tam_train_prepro_nonaug = tam_train_prepro_nonaug_dataset["train"]

tam_dev_prepro_nonaug_dataset = load_dataset("csv", data_files="tam_dev_prepro_nonaug.csv")
tam_dev_prepro_nonaug = tam_dev_prepro_nonaug_dataset["train"]

tam_test_prepro_nonaug_dataset = load_dataset("csv", data_files="tam_test_prepro_nonaug.csv")
tam_test_prepro_nonaug = tam_test_prepro_nonaug_dataset["train"]

print(len(tam_train_prepro_nonaug), len(tam_dev_prepro_nonaug), len(tam_test_prepro_nonaug))

Using custom data configuration default-db822dbb7bcba470


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-db822dbb7bcba470/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-ecfdac70086d763c


...

Using custom data configuration default-d8b923280cf372ba


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d8b923280cf372ba/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

2655 665 829


## Code-mixed

In [10]:
#Reading untouched data (non-preprocessed, non-augmented)

eng_tam_train_nonprepro_nonaug_dataset = load_dataset("csv", data_files="eng_tam_train_nonprepro_nonaug.csv")
eng_tam_train_nonprepro_nonaug = eng_tam_train_nonprepro_nonaug_dataset["train"]

eng_tam_dev_nonprepro_nonaug_dataset = load_dataset("csv", data_files="eng_tam_dev_nonprepro_nonaug.csv")
eng_tam_dev_nonprepro_nonaug = eng_tam_dev_nonprepro_nonaug_dataset["train"]

eng_tam_test_nonprepro_nonaug_dataset = load_dataset("csv", data_files="eng_tam_test_nonprepro_nonaug.csv")
eng_tam_test_nonprepro_nonaug = eng_tam_test_nonprepro_nonaug_dataset["train"]

print(len(eng_tam_train_nonprepro_nonaug), len(eng_tam_dev_nonprepro_nonaug), len(eng_tam_test_nonprepro_nonaug))

Using custom data configuration default-3d985137df3dadd4


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3d985137df3dadd4/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-2a45604fde2c3db1


...

Using custom data configuration default-5e5b257d21c3622c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5e5b257d21c3622c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

3861 965 1207


In [11]:
#Reading preprocessed data (non-augmented)

eng_tam_train_prepro_nonaug_dataset = load_dataset("csv", data_files="eng_tam_train_prepro_nonaug.csv")
eng_tam_train_prepro_nonaug = eng_tam_train_prepro_nonaug_dataset["train"]

eng_tam_dev_prepro_nonaug_dataset = load_dataset("csv", data_files="eng_tam_dev_prepro_nonaug.csv")
eng_tam_dev_prepro_nonaug = eng_tam_dev_prepro_nonaug_dataset["train"]

eng_tam_test_prepro_nonaug_dataset = load_dataset("csv", data_files="eng_tam_test_prepro_nonaug.csv")
eng_tam_test_prepro_nonaug = eng_tam_test_prepro_nonaug_dataset["train"]

print(len(eng_tam_train_prepro_nonaug), len(eng_tam_dev_prepro_nonaug), len(eng_tam_test_prepro_nonaug))

Using custom data configuration default-0dfe0237ad5cfd4d


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0dfe0237ad5cfd4d/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-8810fdf6fa52be6f


...

Using custom data configuration default-5e388b1a6229d793


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5e388b1a6229d793/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

3861 965 1207


# Declaring model name

In [12]:
model_ckpt = "bert-base-multilingual-cased"

# Tokenization

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [14]:
# Define tokenizer function

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

In [15]:
# Ensuring there are no empty records so that tokenization doesn't fail

eng_train_nonprepro_nonaug = eng_train_nonprepro_nonaug.filter(lambda example: example['text'] is not None)
eng_dev_nonprepro_nonaug = eng_dev_nonprepro_nonaug.filter(lambda example: example['text'] is not None)
eng_train_prepro_nonaug = eng_train_prepro_nonaug.filter(lambda example: example['text'] is not None)
eng_dev_prepro_nonaug = eng_dev_prepro_nonaug.filter(lambda example: example['text'] is not None)
eng_train_nonprepro_aug = eng_train_nonprepro_aug.filter(lambda example: example['text'] is not None)
eng_dev_nonprepro_aug = eng_dev_nonprepro_aug.filter(lambda example: example['text'] is not None)

tam_train_nonprepro_nonaug = tam_train_nonprepro_nonaug.filter(lambda example: example['text'] is not None)
tam_dev_nonprepro_nonaug = tam_dev_nonprepro_nonaug.filter(lambda example: example['text'] is not None)
tam_train_prepro_nonaug = tam_train_prepro_nonaug.filter(lambda example: example['text'] is not None)
tam_dev_prepro_nonaug = tam_dev_prepro_nonaug.filter(lambda example: example['text'] is not None)

eng_tam_train_nonprepro_nonaug = eng_tam_train_nonprepro_nonaug.filter(lambda example: example['text'] is not None)
eng_tam_dev_nonprepro_nonaug = eng_tam_dev_nonprepro_nonaug.filter(lambda example: example['text'] is not None)
eng_tam_train_prepro_nonaug = eng_tam_train_prepro_nonaug.filter(lambda example: example['text'] is not None)
eng_tam_dev_prepro_nonaug = eng_tam_dev_prepro_nonaug.filter(lambda example: example['text'] is not None)

print( 
    len(eng_train_nonprepro_nonaug), len(eng_dev_nonprepro_nonaug), len(eng_train_prepro_nonaug), len(eng_dev_prepro_nonaug), \
    len(eng_train_nonprepro_aug), len(eng_dev_nonprepro_aug), len(tam_train_nonprepro_nonaug), len(tam_dev_nonprepro_nonaug), \
    len(tam_train_prepro_nonaug), len(tam_dev_prepro_nonaug), len(eng_tam_train_nonprepro_nonaug), len(eng_tam_dev_nonprepro_nonaug), \
    len(eng_tam_train_prepro_nonaug), len(eng_tam_dev_prepro_nonaug))

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

...

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

3162 791 3162 791 4530 791 2655 665 2655 665 3861 965 3861 965


In [16]:
# Apply tokenizer across all splits in the corpus

eng_train_nonprepro_nonaug_encoded = eng_train_nonprepro_nonaug.map(tokenize, batched=True, batch_size=None)
eng_dev_nonprepro_nonaug_encoded = eng_dev_nonprepro_nonaug.map(tokenize, batched=True, batch_size=None)
eng_train_prepro_nonaug_encoded = eng_train_prepro_nonaug.map(tokenize, batched=True, batch_size=None)
eng_dev_prepro_nonaug_encoded = eng_dev_prepro_nonaug.map(tokenize, batched=True, batch_size=None)
eng_train_nonprepro_aug_encoded = eng_train_nonprepro_aug.map(tokenize, batched=True, batch_size=None)
eng_dev_nonprepro_aug_encoded = eng_dev_nonprepro_aug.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
# Apply tokenizer across all splits in the corpus

tam_train_nonprepro_nonaug_encoded = tam_train_nonprepro_nonaug.map(tokenize, batched=True, batch_size=None)
tam_dev_nonprepro_nonaug_encoded = tam_dev_nonprepro_nonaug.map(tokenize, batched=True, batch_size=None)
tam_train_prepro_nonaug_encoded = tam_train_prepro_nonaug.map(tokenize, batched=True, batch_size=None)
tam_dev_prepro_nonaug_encoded = tam_dev_prepro_nonaug.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
# Apply tokenizer across all splits in the corpus

eng_tam_train_nonprepro_nonaug_encoded = eng_tam_train_nonprepro_nonaug.map(tokenize, batched=True, batch_size=None)
eng_tam_dev_nonprepro_nonaug_encoded = eng_tam_dev_nonprepro_nonaug.map(tokenize, batched=True, batch_size=None)
eng_tam_train_prepro_nonaug_encoded = eng_tam_train_prepro_nonaug.map(tokenize, batched=True, batch_size=None)
eng_tam_dev_prepro_nonaug_encoded = eng_tam_dev_prepro_nonaug.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
print(eng_tam_dev_prepro_nonaug_encoded.column_names)

['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']


# Fine Tuning

In [20]:
#Define performance metrics

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average = "macro")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [21]:
#Extract labels separately

Y_eng_dev_nonprepro_nonaug = np.array(eng_dev_nonprepro_nonaug['label'])
Y_eng_dev_prepro_nonaug = np.array(eng_dev_prepro_nonaug['label'])
Y_eng_dev_nonprepro_aug = np.array(eng_dev_nonprepro_aug['label'])

Y_tam_dev_nonprepro_nonaug = np.array(tam_dev_nonprepro_nonaug['label'])
Y_tam_dev_prepro_nonaug = np.array(tam_dev_prepro_nonaug['label'])

Y_eng_tam_dev_nonprepro_nonaug = np.array(eng_tam_dev_nonprepro_nonaug['label'])
Y_eng_tam_dev_prepro_nonaug = np.array(eng_tam_dev_prepro_nonaug['label'])

## English

### Base model

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
eng_model_base = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                            num_labels=num_labels)
.to(device))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 32
logging_steps = len(eng_train_nonprepro_nonaug_encoded) // batch_size
model_name = f"{model_ckpt}-eng_model_base"
eng_training_args_base = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 3,
    learning_rate = 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    log_level = "error"
)

In [ ]:
trainer = Trainer(
    model = eng_model_base,
    args = eng_training_args_base,
    compute_metrics = compute_metrics,
    train_dataset = eng_train_nonprepro_nonaug_encoded,
    eval_dataset = eng_dev_nonprepro_nonaug_encoded,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.246600,0.264636,0.924147,0.320193
2,0.166600,0.228305,0.926675,0.371069
3,0.125100,0.229952,0.930468,0.425032


TrainOutput(global_step=297, training_loss=0.17889929098713678, metrics={'train_runtime': 260.9782, 'train_samples_per_second': 36.348, 'train_steps_per_second': 1.138, 'total_flos': 1247946940283904.0, 'train_loss': 0.17889929098713678, 'epoch': 3.0})

In [ ]:
eng_preds_output_base = trainer.predict(eng_dev_nonprepro_nonaug_encoded)

In [ ]:
eng_y_preds_base = np.argmax(eng_preds_output_base.predictions, axis=1)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_eng_dev_nonprepro_nonaug, eng_y_preds_base, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.63      0.21      0.31        58
Non-anti LGBTQIA+ content       0.94      0.99      0.96       731
              transphobic       0.00      0.00      0.00         2

                 accuracy                           0.93       791
                macro avg       0.52      0.40      0.43       791
             weighted avg       0.91      0.93      0.91       791



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Base + preprocessing model

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
eng_model_base_prepro = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                            num_labels=num_labels)
.to(device))

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(eng_train_prepro_nonaug_encoded) // batch_size
model_name = f"{model_ckpt}-eng_model_base_prepro"
eng_training_args_base_prepro = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 3,
    learning_rate = 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    log_level = "error"
)

In [ ]:
trainer = Trainer(
    model = eng_model_base_prepro,
    args = eng_training_args_base_prepro,
    compute_metrics = compute_metrics,
    train_dataset = eng_train_prepro_nonaug_encoded,
    eval_dataset = eng_dev_prepro_nonaug_encoded,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.244800,0.287820,0.924147,0.320193
2,0.201800,0.302987,0.924147,0.320193
3,0.155600,0.354737,0.927939,0.388392


TrainOutput(global_step=1188, training_loss=0.20043004276575865, metrics={'train_runtime': 439.385, 'train_samples_per_second': 21.589, 'train_steps_per_second': 2.704, 'total_flos': 1652554737329076.0, 'train_loss': 0.20043004276575865, 'epoch': 3.0})

In [ ]:
eng_preds_output_base_prepro = trainer.predict(eng_dev_prepro_nonaug_encoded)

In [ ]:
eng_y_preds_base_prepro = np.argmax(eng_preds_output_base_prepro.predictions, axis=1)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_eng_dev_prepro_nonaug, eng_y_preds_base_prepro, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.64      0.12      0.20        58
Non-anti LGBTQIA+ content       0.93      0.99      0.96       731
              transphobic       0.00      0.00      0.00         2

                 accuracy                           0.93       791
                macro avg       0.52      0.37      0.39       791
             weighted avg       0.91      0.93      0.90       791



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Base + augmentation model

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
eng_model_base_aug = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                            num_labels=num_labels)
.to(device))

In [ ]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 32
logging_steps = len(eng_train_nonprepro_aug_encoded) // batch_size
model_name = f"{model_ckpt}-eng_model_base_aug"
eng_training_args_base_aug = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 3,
    learning_rate = 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    log_level = "error"
)

In [ ]:
trainer = Trainer(
    model = eng_model_base_aug,
    args = eng_training_args_base_aug,
    compute_metrics = compute_metrics,
    train_dataset = eng_train_nonprepro_aug_encoded,
    eval_dataset = eng_dev_nonprepro_aug_encoded,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.313700,0.310486,0.912769,0.361321
2,0.103200,0.298334,0.915297,0.445176
3,0.057200,0.340600,0.908976,0.439331


TrainOutput(global_step=426, training_loss=0.15697393543196256, metrics={'train_runtime': 623.3218, 'train_samples_per_second': 21.803, 'train_steps_per_second': 0.683, 'total_flos': 1641195637748100.0, 'train_loss': 0.15697393543196256, 'epoch': 3.0})

In [ ]:
eng_preds_output_base_aug = trainer.predict(eng_dev_nonprepro_aug_encoded)

In [ ]:
eng_y_preds_base_aug = np.argmax(eng_preds_output_base_aug.predictions, axis=1)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_eng_dev_nonprepro_aug, eng_y_preds_base_aug, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.39      0.34      0.37        58
Non-anti LGBTQIA+ content       0.95      0.96      0.95       731
              transphobic       0.00      0.00      0.00         2

                 accuracy                           0.91       791
                macro avg       0.45      0.43      0.44       791
             weighted avg       0.90      0.91      0.91       791



## Tamil

### Base model

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
tam_model_base = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                            num_labels=num_labels)
.to(device))

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(tam_train_nonprepro_nonaug_encoded) // batch_size
model_name = f"{model_ckpt}-tam_model_base"
tam_training_args_base = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 3,
    learning_rate = 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    log_level = "error"
)

In [ ]:
trainer = Trainer(
    model = tam_model_base,
    args = tam_training_args_base,
    compute_metrics = compute_metrics,
    train_dataset = tam_train_nonprepro_nonaug_encoded,
    eval_dataset = tam_dev_nonprepro_nonaug_encoded,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.648500,0.494638,0.827068,0.437773
2,0.453400,0.401796,0.863158,0.679653
3,0.299000,0.460561,0.870677,0.736495


TrainOutput(global_step=996, training_loss=0.46676965315179175, metrics={'train_runtime': 518.2056, 'train_samples_per_second': 15.37, 'train_steps_per_second': 1.922, 'total_flos': 2095698372203520.0, 'train_loss': 0.46676965315179175, 'epoch': 3.0})

In [ ]:
ta_preds_output_base = trainer.predict(tam_dev_nonprepro_nonaug_encoded)

In [ ]:
tam_y_preds_base = np.argmax(ta_preds_output_base.predictions, axis=1)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_tam_dev_nonprepro_nonaug, tam_y_preds_base, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.63      0.80      0.70       103
Non-anti LGBTQIA+ content       0.95      0.91      0.93       526
              transphobic       0.61      0.56      0.58        36

                 accuracy                           0.87       665
                macro avg       0.73      0.75      0.74       665
             weighted avg       0.88      0.87      0.87       665



### Base + preprocessing model

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tam_model_base_prepro = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                            num_labels=num_labels)
.to(device))

In [ ]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(tam_train_prepro_nonaug_encoded) // batch_size
model_name = f"{model_ckpt}-tam_model_base_prepro"
tam_training_args_base_prepro = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 3,
    learning_rate = 2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    log_level = "error"
)

In [ ]:
trainer = Trainer(
    model = tam_model_base_prepro,
    args = tam_training_args_base_prepro,
    compute_metrics = compute_metrics,
    train_dataset = tam_train_prepro_nonaug_encoded,
    eval_dataset = tam_dev_prepro_nonaug_encoded,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.631900,0.443274,0.846617,0.508238
2,0.443900,0.409760,0.864662,0.688151
3,0.282400,0.475937,0.864662,0.715028


TrainOutput(global_step=996, training_loss=0.45239745852458907, metrics={'train_runtime': 518.3205, 'train_samples_per_second': 15.367, 'train_steps_per_second': 1.922, 'total_flos': 2095698372203520.0, 'train_loss': 0.45239745852458907, 'epoch': 3.0})

In [ ]:
tam_preds_output_base_prepro = trainer.predict(tam_dev_prepro_nonaug_encoded)

In [ ]:
tam_y_preds_base_prepro = np.argmax(tam_preds_output_base_prepro.predictions, axis=1)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_tam_dev_prepro_nonaug, tam_y_preds_base_prepro, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.64      0.79      0.71       103
Non-anti LGBTQIA+ content       0.94      0.90      0.92       526
              transphobic       0.53      0.50      0.51        36

                 accuracy                           0.86       665
                macro avg       0.70      0.73      0.72       665
             weighted avg       0.87      0.86      0.87       665



## C-M

### Base model

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
eng_tam_model_base = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                            num_labels=num_labels)
.to(device))

In [ ]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(eng_tam_train_nonprepro_nonaug_encoded) // batch_size
model_name = f"{model_ckpt}-eng_tam_model_base"
eng_tam_training_args_base = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 5,
    learning_rate = 3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    log_level = "error"
)

In [ ]:
trainer = Trainer(
    model = eng_tam_model_base,
    args = eng_tam_training_args_base,
    compute_metrics = compute_metrics,
    train_dataset = eng_tam_train_nonprepro_nonaug_encoded,
    eval_dataset = eng_tam_dev_nonprepro_nonaug_encoded,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.434200,0.420712,0.892228,0.314348
2,0.420500,0.449670,0.892228,0.314348
3,0.432700,0.435976,0.892228,0.314348
4,0.410500,0.397042,0.892228,0.314348
5,0.363500,0.394845,0.894301,0.426012


TrainOutput(global_step=2415, training_loss=0.41226713366143203, metrics={'train_runtime': 1151.7893, 'train_samples_per_second': 16.761, 'train_steps_per_second': 2.097, 'total_flos': 4593289642651710.0, 'train_loss': 0.41226713366143203, 'epoch': 5.0})

In [ ]:
eng_tam_preds_output_base = trainer.predict(eng_tam_dev_nonprepro_nonaug_encoded)

In [ ]:
eng_tam_y_preds_base = np.argmax(eng_tam_preds_output_base.predictions, axis=1)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_eng_tam_dev_nonprepro_nonaug, eng_tam_y_preds_base, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.38      0.29      0.33        66
Non-anti LGBTQIA+ content       0.92      0.98      0.95       861
              transphobic       0.00      0.00      0.00        38

                 accuracy                           0.89       965
                macro avg       0.43      0.42      0.43       965
             weighted avg       0.85      0.89      0.87       965



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Base + preprocessing model

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
eng_tam_model_base_prepro = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                            num_labels=num_labels)
.to(device))

In [ ]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(eng_tam_train_prepro_nonaug_encoded) // batch_size
model_name = f"{model_ckpt}-eng_tam_model_base_prepro"
eng_tam_training_args_base_prepro = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 5,
    learning_rate = 3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    log_level = "error"
)

In [ ]:
trainer = Trainer(
    model = eng_tam_model_base_prepro,
    args = eng_tam_training_args_base_prepro,
    compute_metrics = compute_metrics,
    train_dataset = eng_tam_train_prepro_nonaug_encoded,
    eval_dataset = eng_tam_dev_prepro_nonaug_encoded,
    tokenizer = tokenizer
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.425500,0.430954,0.892228,0.314348


In [ ]:
eng_tam_preds_output_base_prepro = trainer.predict(eng_tam_dev_prepro_nonaug_encoded)

In [ ]:
eng_tam_y_preds_base_prepro = np.argmax(eng_tam_preds_output_base_prepro.predictions, axis=1)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_eng_tam_dev_prepro_nonaug, eng_tam_y_preds_base_prepro, target_names=target_names))